<a href="https://colab.research.google.com/github/amien1410/colab-notebooks/blob/main/Colab_Pyspark_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install pyspark
!pip install pyspark

# initiate pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# download sample
!wget -O Mall_Customers.csv "https://raw.githubusercontent.com/Shahadat9886/Mall_Customers.csv/refs/heads/main/Thu%20H%C3%A0%20Tr%E1%BA%A7n%20-%20Mall_Customers.csv"

# load csv
df = spark.read.csv("/content/Mall_Customers.csv", header=True, inferSchema=True)
df

--2025-06-10 05:59:32--  https://raw.githubusercontent.com/Shahadat9886/Mall_Customers.csv/refs/heads/main/Thu%20H%C3%A0%20Tr%E1%BA%A7n%20-%20Mall_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4286 (4.2K) [text/plain]
Saving to: ‘Mall_Customers.csv’

Mall_Customers.csv  100%[===================>]   4.19K  --.-KB/s    in 0s      

2025-06-10 05:59:33 (49.8 MB/s) - ‘Mall_Customers.csv’ saved [4286/4286]



DataFrame[CustomerID: int, Genre: string, Age: int, Annual Income (k$): int, Spending Score (1-100): int]

In [2]:
# use spark.sql
df2 = spark.sql("select 'spark' as hello ")
df2.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [4]:
# !wget -O movies.csv "https://raw.githubusercontent.com/Apress/applied-data-science-using-pyspark/refs/heads/main/Ch02/Chapter2_Data/movie_data_part1.csv"
# df = spark.read.csv("movies.csv", header=True, sep=',')
df.show(10)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|belongs_to_collection|budget|id|original_language|original_title|overview|popularity|production_companies|production_countries|release_date|revenue|runtime|status|tagline|title|vote_average|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                         |0|43000|fr|Le Ca...|
|                                                                                                                                                                         |0|43001|fr|Cybèl...|
|                                       

In [5]:
# Importing the SparkContext module from PySpark
from pyspark import SparkContext

# Creating a SparkContext object
sc = SparkContext.getOrCreate()

# Importing the random module
import random
NUM_SAMPLES = 100000000 # Define the samples

# Function to check if a point lies inside
def inside(p):
  x, y = random.random(), random.random()
  return x*x + y*y < 1

# parallelize the computation
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()

# calculate the estimate value of pi
pi = 4 * count / NUM_SAMPLES
print("Pi is roughly", pi)

Pi is roughly 3.14192564
